In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn.decomposition import TruncatedSVD,SparsePCA
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,recall_score

import gc
import time
import os
import sys
import warnings
warnings.filterwarnings('ignore')

In [3]:
pickle_path = "../pickle"
active = pd.read_pickle("{}/user_app_active.pickle".format(pickle_path))
usage = pd.read_pickle("{}/user_app_seq.pickle".format(pickle_path))

In [5]:
vector_path = "../vector"
from glove import *

for i,df in enumerate([active,usage]): # 预计耗时 222 + 2100s
    t1 = time.time()
    c = Corpus()
    c.fit(df['appid'].values)
    glove = Glove(no_components=300, learning_rate=0.05) 
    glove.fit(c.matrix,epochs=12,no_threads=30,verbose=1)
    glove.add_dictionary(c.dictionary)
    glove.save("{}/{}_glove300.model".format(vector_path,i))
    print(time.time()-t1)

In [ ]:
# Gen W2V Vector
from gensim import models

for i,df in enumerate([active,usage]): # 预计耗时 1511 + 5874
    t1 = time.time()
    w2v = models.Word2Vec(df['appid'].values, size=300, window=20, workers=40,hs=1) # 设置sg的话 变成skip-gram方法 我们测试效果差不多
    w2v.wv.save_word2vec_format("{}/{}_w2v300.model".format(vector_path,i))
    print(time.time()-t1)


1511.6713757514954


In [ ]:
t = []
c = []
for i,df in enumerate([active,usage]): # 预计耗时 200 + 750s
    t1 = time.time()
    tfidf = TfidfVectorizer(analyzer='word',token_pattern=u"(?u)\\b\\w+\\b",min_df=1,ngram_range=(1,1))
    t.append(tfidf.fit_transform(df['appid'].map(lambda x:' '.join(x)).values))
    cv = CountVectorizer(analyzer='word',token_pattern=u"(?u)\\b\\w+\\b",min_df=1,ngram_range=(1,1))
    c.append(cv.fit_transform(df['appid'].map(lambda x:' '.join(x)).values))
    
    print(time.time()-t1)

from scipy import sparse
if not os.path.exists("{}/Sparse_Matrix".format(vector_path)):
    os.mkdir("{}/Sparse_Matrix".format(vector_path))
sparse.save_npz('{}/Sparse_Matrix/active_tfidf.npz'.format(vector_path), t[0])
sparse.save_npz('{}/Sparse_Matrix/usage_tfidf.npz'.format(vector_path), t[1])

sparse.save_npz('{}/Sparse_Matrix/active_count.npz'.format(vector_path), c[0])
sparse.save_npz('{}/Sparse_Matrix/usage_count.npz'.format(vector_path), c[1])